In [2]:
# Importing relevant librairies
import numpy as np
import pandas as pd
import yfinance as yf
import os
pd.set_option('display.max_columns', None)

In [12]:
def load_csv(ticker: str):
    """
    Creating a function that would allow us to load a csv containing all relevant features of a given ticker
    ticker should be in capital
    Start and end should be in the be following format: %YYYY-%MM-%DD
    """

    #Loading all relevant csv files

    #Loading stock return
    ticker_df = pd.read_csv(f'../raw_data/stocks_return/adj_return_{ticker}.csv')
    ticker_df.rename(columns={'timestamp':'date'}, inplace=True)
    ticker_df.sort_values('date', ascending=True, inplace=True)
    ticker_df.reset_index(inplace=True, drop=True)

    if '1999-12-31' in ticker_df['date'].values:
        ticker_df = ticker_df[ticker_df['date']>'1999-12-31']
        ticker_df.reset_index(inplace=True, drop=True)

    #Loading stocks eps
    ticker_eps = pd.read_csv(f'../raw_data/eps/data_{ticker}.csv', index_col=0)
    ticker_eps['date'] = ticker_eps['reportedDate'].copy()
    ticker_eps.sort_values('date', inplace=True)
    ticker_eps['reportedDate']= pd.to_datetime(ticker_eps['reportedDate'])
    ticker_eps['year'] = pd.DatetimeIndex(ticker_eps['reportedDate']).year

    if 2000 in ticker_eps['year'].values:
        ticker_eps = ticker_eps[ticker_eps['year']>=2000]
        ticker_eps.reset_index(inplace=True, drop=True)

    ticker_eps.drop(columns=['year','fiscalDateEnding', 'estimatedEPS', 'surprise', 'reportedDate'], inplace=True)

    #Loading all macros and commodities
    gold = pd.read_csv('../raw_data/macro/gold.csv', index_col=0)
    us_dollar = pd.read_csv('../raw_data/macro/usd.csv', index_col=0)

    credit_spread = pd.read_csv('../raw_data/macro/us_yields.csv', index_col=0)
    oil = pd.read_csv('../raw_data/macro/oil.csv', index_col=0)
    orders = pd.read_csv('../raw_data/macro/orders.csv', index_col=0)
    ffunds_rate = pd.read_csv('../raw_data/macro/fed_funds.csv', index_col=0)
    unemployment = pd.read_csv('../raw_data/macro/unemployment.csv', index_col=0)
    inflation_expectation = pd.read_csv('../raw_data/macro/inflation_expectation.csv', index_col=0)
    non_farm_payroll = pd.read_csv('../raw_data/macro/non_farm_payroll.csv', index_col=0)
    cpi = pd.read_csv('../raw_data/macro/cpi.csv', index_col=0)
    retails = pd.read_csv('../raw_data/macro/retail_sales.csv', index_col=0)
    gdp_capita = pd.read_csv('../raw_data/macro/gdp_per_capita.csv', index_col=0)


    #Creating our final df and merging with relevant files
    final_df = ticker_df.copy()
    final_df = final_df.merge(ticker_eps, how='outer', on='date')
    final_df = final_df.merge(credit_spread, how='outer')
    final_df = final_df.merge(oil, how='outer', on='date')
    final_df = final_df.merge(orders, how='outer')
    final_df = final_df.merge(ffunds_rate, how='outer')
    final_df = final_df.merge(unemployment, how='outer')
    final_df = final_df.merge(inflation_expectation, how='outer')
    final_df = final_df.merge(non_farm_payroll, how='outer')
    final_df = final_df.merge(cpi, how='outer')
    final_df = final_df.merge(retails, how='outer')
    final_df = final_df.merge(gdp_capita, how='outer')
    final_df = final_df.merge(gold, how='outer', on='date')
    final_df = final_df.merge(us_dollar, how='outer', on='date')

    #Sorting by chronological order and resetting index
    final_df = final_df.sort_values('date', ascending=True)
    final_df.reset_index(drop=True, inplace=True)

    #Since not all features have daily data, we forward filled the missing values
    final_df['orders'] = final_df['orders'].fillna(method='ffill')
    final_df['retail_sales'] = final_df['retail_sales'].fillna(method='ffill')
    final_df['gdp_per_capita'] = final_df['gdp_per_capita'].fillna(method='ffill')
    final_df['CPI'] = final_df['CPI'].fillna(method='ffill')
    final_df['non_farm_payroll'] = final_df['non_farm_payroll'].fillna(method='ffill')
    final_df['inf_exp'] = final_df['inf_exp'].fillna(method='ffill')
    final_df['unemployment_rate'] = final_df['unemployment_rate'].fillna(method='ffill')
    final_df['reportedEPS'] = final_df['reportedEPS'].fillna(method='ffill')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace(['None'], '0')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace([np.nan], '0')
    
    #Dropping null values
    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    
    # calculating the return
    final_df['return'] = final_df['adjusted_close'].pct_change()
    final_df['return'][0]='0'

    return final_df

In [13]:
# reportedEPS etaler sur toutes les dates
# surprise percentage mettre a 0
# dans le pipe il faut pas toucher au targert ?

In [14]:
apple = load_csv('AAPL')
apple

/tmp/ipykernel_12166/842115558.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]='0'


,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return
0,2000-01-19,105.620,108.75,103.370,106.56,0.810916,5336100.0,0.0,1.0,0.04,33.3333,0.0673,0.0646,0.0027,29.11,185813.0,0.0547,4.0,0.03,128992.0,168.800,213709.0,45983.0,"289,15",101.760002,0
1,2000-01-20,115.500,121.50,113.500,113.50,0.863729,16349400.0,0.0,1.0,0.04,0,0.0679,0.0649,0.0030,29.67,185813.0,0.0544,4.0,0.03,128992.0,168.800,213709.0,45983.0,"288,25",101.360001,0.065128
2,2000-01-21,114.250,114.25,110.190,111.31,0.847063,4427900.0,0.0,1.0,0.04,0,0.0679,0.0648,0.0031,29.71,185813.0,0.0536,4.0,0.03,128992.0,168.800,213709.0,45983.0,"288,7",101.760002,-0.019295
3,2000-01-24,108.440,112.75,105.120,106.25,0.808557,3936400.0,0.0,1.0,0.04,0,0.0669,0.0643,0.0026,29.25,185813.0,0.0553,4.0,0.03,128992.0,168.800,213709.0,45983.0,"287,4",102.199997,-0.045459
4,2000-01-25,105.000,113.12,102.370,112.25,0.854216,4438800.0,0.0,1.0,0.04,0,0.0670,0.0644,0.0026,30.28,185813.0,0.0546,4.0,0.03,128992.0,168.800,213709.0,45983.0,"285,6",102.410004,0.056471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,2022-11-16,149.130,149.87,147.290,148.79,148.790000,64218266.0,0.0,1.0,1.29,0,0.0367,0.0435,-0.0068,85.62,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,"1773,77",106.279999,-0.008331
5682,2022-11-17,146.430,151.48,146.150,150.72,150.720000,80389400.0,0.0,1.0,1.29,0,0.0377,0.0443,-0.0066,81.69,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,"1760,87",106.690002,0.012971
5683,2022-11-18,152.305,152.70,149.970,151.29,151.290000,74829573.0,0.0,1.0,1.29,0,0.0382,0.0451,-0.0069,80.07,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,"1749,74",106.930000,0.003782
5684,2022-11-21,150.160,150.37,147.715,148.01,148.010000,58724070.0,0.0,1.0,1.29,0,0.0383,0.0448,-0.0065,79.74,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,"1737,8081",107.839996,-0.02168


In [196]:
apple['surprisePercentage'] = apple['surprisePercentage'].replace(['None'], '0')

In [176]:
apple['surprisePercentage'] = apple['surprisePercentage'].replace([np.nan], '0')

In [181]:
apple['surprisePercentage'].unique()

array(['0', '33.3333', '100', '66.6667', '25', '16.6667', '28.5714',
       '45.4545', '30', '8.6957', '13.3333', '12.5', '18.75', '11.7647',
       '73.3333', '37.1429', '13.6364', '13.7931', '19.4805', '18.1818',
       '33.7349', '-4.717', '36.5517', '23.0769', '-10.1351', '-0.8',
       '2.6042', '0.6993', '1.9048', '3.5088', '2.9851', '13.6986',
       '4.065', '8.3969', '17.6923', '7.8704', '2.2099', '4.2553',
       '1.548', '-5', '2.8986', '0.6024', '4.6729', '3.9604', '6.3694',
       '10.6952', '0.7772', '1.487', '7.3394', '4.2857', '0.9615',
       '5.0847', '3.7736', '7.0423', '9.6491', '14.2857', '27.451',
       '19.1489', '41.4141', '28.7129', '11.1111', '6.2937', '3.4483',
       '1.5748'], dtype=object)